При подготовке использовались [материалы](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/2020/nn_intro_torch.ipynb) курса Михаила Нефедова

In [ ]:
!pip install ipdb

In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import ipdb

### Классификация твитов по тональности



#### Скачивание и подготовка данных

In [ ]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

In [ ]:
pos_tweets = pd.read_csv('positive.csv', encoding='utf-8', sep=';', header=None,  names=[0, 1, 2, 'text', 'tone', 5, 6, 7, 8, 9, 10, 11])

In [ ]:
neg_tweets = pd.read_csv('negative.csv', encoding='utf-8', sep=';', header=None, names=[0, 1, 2, 'text', 'tone', 5, 6, 7, 8, 9, 10, 11])
neg_tweets['tone'] = 0

In [ ]:
all_tweets_data = pos_tweets.append(neg_tweets)
print(len(all_tweets_data))

In [ ]:
tweets_data = shuffle(all_tweets_data[['text', 'tone']])[:100000]

In [ ]:
train_sentences, val_sentences = train_test_split(tweets_data, test_size=0.1)

In [ ]:
train_sentences[:10]

In [ ]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

Теперь нам нужно собрать все уникальные слова в словарь. Лучше сразу посчитать количество упоминаний, чтобы отсеять самые редкие.

In [ ]:
vocab = Counter()

for text in tweets_data['text']:
    vocab.update(preprocess(text))
print('всего уникальных токенов:', len(vocab))

In [ ]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('уникальных токенов, втретившихся больше 2 раз:', len(filtered_vocab))

In [ ]:
#создаем словарь с индексами word2id, для спецсимвола паддинга дефолтный индекс - 0
word2id = {'PAD': 0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [ ]:
#обратный словарь для того, чтобы раскодировать последовательность
id2word = {i: word for word, i in word2id.items()}

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE


#### паддинги

In [ ]:
MAX_LEN = 0

for text in tweets_data.text:
    tokens = preprocess(text)
    MAX_LEN = max(len(tokens), MAX_LEN)
MAX_LEN

##### F.pad 
паддим каждую последовательность до желаемой длины

In [ ]:
X = torch.LongTensor(size=(train_sentences.shape[0], MAX_LEN))

for i, text in enumerate(train_sentences.text):
    tokens = preprocess(text) # токенизируем
    
    ids = [word2id[token] for token in tokens if token in word2id][:MAX_LEN]

    ids = F.pad(torch.LongTensor(ids), (0, MAX_LEN - len(ids)))
    X[i] = ids

In [ ]:
print(X[4].shape)
print(X[4])
print([id2word[int(id_)] for id_ in  X[4]])

##### torch.nn.utils.rnn.pad_sequence
альтернатива - добиваем паддингами до самого длинного предложения из переданных

In [ ]:
X = []

for text in train_sentences.text:
    tokens = preprocess(text) 
    ids = torch.LongTensor([word2id[token] for token in tokens if token in word2id]) 
    X.append(ids)

In [ ]:
X = pad_sequence(X, batch_first=True) 
X.shape

In [ ]:
X = X[:, :5] # если хочется обрезать короче

In [ ]:
X.shape

#### Dataset & DataLoader

In [ ]:
class TweetsDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['tone'].values
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        tokens = self.preprocess(self.dataset[index]) # токенизируем
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return ids, y
    
    def preprocess(self, text):
        tokens = text.lower().split()
        tokens = [token.strip(punctuation) for token in tokens]
        tokens = [token for token in tokens if token]
        return tokens

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, y = list(zip(*batch))
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      #мы хотим применять BCELoss, он будет брать на вход predicted размера batch_size x 1 
      #(так как для каждого семпла модель будет отдавать одно число), target размера batch_size x 1
      y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]] 
      return padded_ids, y

##### создаем итераторы по данным для трейна и теста

In [ ]:
train_dataset = TweetsDataset(train_sentences, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)

In [ ]:
len(train_iterator)

In [ ]:
batch = next(iter(train_iterator))

In [ ]:
batch

In [ ]:
val_dataset = TweetsDataset(val_sentences, word2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn=val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)

In [ ]:
test_batch = next(iter(val_iterator))

#### Embedding  слой
перед тем как собрать сеть целиком, посмотрим на слои, которые пока не обсуждались

In [ ]:
layer = nn.Embedding(10, 5) # первый параметр  - размер всего словаря, второй параметр размер получаемого эмбеддинга

In [ ]:
layer.weight

In [ ]:
input = torch.tensor([1, 4, 3, 3, 7, 0, 9])
result = layer(input)
result

#### Dropout

In [ ]:
dropout = nn.Dropout(p=0.5)

In [ ]:
result

In [ ]:
dropout(result)

#### MLP model

In [ ]:
class MLP(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        
        super().__init__()          
        # указываем в атрибутах класса, какие слои и активации нам понадобятся
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.emb2h = nn.Linear(embedding_dim, 10) 
        self.act1 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.h2out = nn.Linear(10, 1)
        self.act2 = nn.Sigmoid()
        
        
    def forward(self, text): #необходимый метод,  в нем указываем, как именно связываются слои/активации между собой
        # batch_size x seq_len
        #ipdb.set_trace()
        embedded = self.embedding(text)   # переводим последовательность индексов в последовательность эмбеддингов
        # batch_size x seq_len x embedding_dim
        
        mean_emb = torch.mean(embedded, dim=1) # считаем средний эмбеддинг предложения
        # batch_size x embedding_dim
        hidden = self.emb2h(mean_emb) # пропускаем эмбеддинг через полносвязный слой 
        # batch_size x 10
        hidden = self.act1(hidden)
        # batch_size x 10
        hidden = self.dropout(hidden)
        # batch_size x 10
        out = self.h2out(hidden) # возвращаем одно число для каждого семпла
        # batch_size x 1
        proba = self.act2(out) # пропускаем число через сигмоиду, делая из него вероятность класса
        # batch_size x 1
        return proba

In [ ]:
batch, y = next(iter(train_iterator))
batch, y = batch.to(device='cpu'), y.to(device='cpu')
print(batch.shape)
print(y.shape)

In [ ]:
y

In [ ]:
#пропустим через модель наш первый батч, чтобы проверить, что все работает
model = MLP(len(id2word), 5)
output = model(batch)
output

In [ ]:
loss = nn.BCELoss()
loss(output, y)

In [ ]:
model.state_dict() # где посмотреть веса модели (ее параметры)

#### training loop, логика обучения и валидации

теперь нам нужны функции для обучения и валидации,
каждый вызов функции - одна эпоха обучения 

За одну эпоху нам надо для каждого батча:

-- применить к нему модель, 

-- посчитать значение функции потерь, 

-- посчитать градиенты,

-- обновить веса (параметры модели)


In [ ]:
def train(model, iterator, optimizer, criterion):
    print('Training...')
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах
    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds_proba = model(texts) #прогоняем данные через модель
        loss = criterion(preds_proba, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        
        if not (i + 1) % 20:
            print(f'Train loss: {epoch_loss/i}')
        
    return epoch_loss / len(iterator) # возвращаем среднее значение функции потерь по всей выборке

In [ ]:
def evaluate(model, iterator, criterion):
    print("\nValidating...")
    epoch_loss = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):   
            predictions = model(texts)  # делаем предсказания на тесте
            loss = criterion(predictions, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item() 
            if not (i + 1) % 5:
              print(f'Val loss: {epoch_loss/i}')
        
    return epoch_loss / len(iterator) # возвращаем средний лосс по батчам

#### инициализируем модель, задаем оптимизатор и функцию потерь

In [ ]:
model = MLP(len(word2id), 5)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss() # Binary Cross Entropy

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

#### запуск обучения!

In [ ]:
losses = []
losses_eval = []

for i in range(20):
    print(f'\nstarting Epoch {i}')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)

    epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.plot(losses_eval)
plt.title('BCE loss value')
plt.ylabel('BCE loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

# как инициализировать модель готовыми эмбеддингами

In [ ]:
import gensim

In [ ]:
texts = all_tweets_data.text.apply(preprocess).tolist()

In [ ]:
len(texts)

In [ ]:
%%time
w2v = gensim.models.Word2Vec(texts, size=100, window=5, min_count=1)

In [ ]:
w2v.wv.most_similar('веселый')


In [ ]:
weights = np.zeros((len(word2id), 100))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue   
    try:
        weights[i] = w2v.wv[word]    
    except KeyError:
      count += 1
      # oov словам сопоставляем случайный вектор
      weights[i] = np.random.normal(0,0.1,100)

In [ ]:
class MLP_w2v(nn.Module):
    
    def __init__(self, vocab_size):
        
        super().__init__()          
        # указываем в атрибутах класса, какие слои и активации нам понадобятся
        self.embedding = nn.Embedding(vocab_size, 100)
        self.embedding.from_pretrained(torch.tensor(weights), freeze=True)
        self.emb2h = nn.Linear(100, 10) 
        self.act1 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.h2out = nn.Linear(10, 1)
        self.act2 = nn.Sigmoid() 
        
        
    def forward(self, text): #необходимый метод,  в нем указываем, как именно связываются слои/активации между собой
        
        embedded = self.embedding(text)   # переводим последовательность индексов в последовательность эмбеддингов
        mean_emb = torch.mean(embedded, dim=1) # считаем средний эмбеддинг предложения
        hidden = self.emb2h(mean_emb) # пропускаем эмбеддинг через полносвязный слой 
        hidden = self.dropout(hidden)
        hidden = self.act1(hidden)
        hidden = self.dropout(hidden)
        out = self.h2out(hidden) # возвращаем одно число для каждого семла
        proba = self.act2(out) # пропускаем число через сигмоиду, делая из него вероятность класса
        
        return proba

In [ ]:
model = MLP_w2v(len(word2id))
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss() # Binary Cross Entropy

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [ ]:
losses = []
losses_eval = []

for i in range(12):
    print(f'\nstarting Epoch {i}')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)

    epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.plot(losses_eval)
plt.title('BCE loss value')
plt.ylabel('BCE loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()